In [53]:
import sqlite3
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import time
from itertools import combinations

In [54]:
# Parameters

print("Pick Database: ")
print("1 - Amazon Books")
print("2 - Best Buy")
print("3 - K-mart")
print("4 - Nike")
print("5 - Generic")
print("6 - Homework Example")
print("7 - Amazon Food")

db = str(input("Please input the desired database"))
min_support = float(input("Please input the desired minimum support"))
min_confidence = float(input("Please input the desired minimum confidence"))


Pick Database: 
1 - Amazon Books
2 - Best Buy
3 - K-mart
4 - Nike
5 - Generic
6 - Homework Example
7 - Amazon Food


In [55]:
dic = {
    '1': "amazon_book",
    '2': "bestbuy",
    '3': 'kmart',
    '4': 'nike',
    '5': 'generic',
    '6': 'class_example',
    '7': 'amazon_Food'
}

table = dic[db]
print("Selected database:", table)
print("Selected minimum support:", min_support)
print("Selected minimum confidence:", min_confidence)

Selected database: amazon_Food
Selected minimum support: 0.2
Selected minimum confidence: 0.2


Brute Force Section

In [56]:
def brute_force(table):
    # Connect to SQLite database
    conn = sqlite3.connect(f'{table}.db')

    # Execute SELECT query to retrieve data from the table
    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)

    # Close the connection
    conn.close()

    
    df['Transactions'] = df['Transactions'].str.split(', ').apply(set)

    def generate_combinations(lst, k):
        return set(combinations(lst, k))

    def calculate_min_support(data, itemset):
        count = 0
        for row in data:
            if set(item for tpl in itemset for item in tpl).issubset(row):
                count += 1
        return count / df.shape[0]

    # Generate all frequent 1-itemset
    def first_pass(df, support):
        item = set()
        max_transaction_length = 1
        # Find all unique items in the transaction list and max transaction length
        for index, row in df.iterrows():
            for i in row['Transactions']:
                item.add(tuple([i]))
                max_transaction_length = max(len(row['Transactions']), max_transaction_length)

        itemsets = []

        # Add all 1-itemset and support to the itemsets
        for i in item:
            support = calculate_min_support(df['Transactions'].values, tuple([i]))
            itemsets.append([[i], support])

        return item, itemsets, max_transaction_length

    # Generate all k-itemset, where k is between 2 and the max transaction length
    def second_pass(df, item, itemsets, support, max_transaction_length):
        k = 2
        
        # The stop condition for brute force will be the k-itemset where k is the max transaction length
        while k <= max_transaction_length:

            # Generate and add all k-itemset and support to the itemsets
            all_combinations = generate_combinations(item, k)
            for i in all_combinations:
                support = calculate_min_support(df['Transactions'].values, i)
                itemsets.append([list(i), support])
            k += 1
            
        return itemsets

    def filter_itemsets(itemsets, min_support):
        res = []
        for i in itemsets:
            subset, support = i
            if support >= min_support:
                res.append(i)
        return res

    # Begin timer
    start = time.time()

    item, itemsets, max_transaction_length = first_pass(df, min_support)
    if max_transaction_length >= 2:
        itemsets = second_pass(df, item, itemsets, min_support, max_transaction_length)
    frequent_itemsets = filter_itemsets(itemsets, min_support)

    # Create all subset for an item in the frequent itemset
    # If item = A, B and item length is 2
    # The subsets would be A, B, AB
    def find_subset(item, item_length):
        subsets = []
        for i in range(1, item_length + 1):
            subsets.extend(combinations(item, i))
        return subsets

    # Generate all association rules from the frequent itemset
    def association_rules(frequent_itemsets, min_confidence):
        rules = list()
        dic = {}

        # Convert the frequent itemset into a dictionary
        for item in frequent_itemsets:
            key = frozenset(item[0])
            value = item[1]
            dic[key] = value

        # Generate all possible {X} -> {Y} and checks if {XY} exist to compute confidence
        for item, support in dic.items():
            item_length = len(item)
            if item_length > 1:
                subsets = find_subset(item, item_length)
                for X in subsets:
                    Y = set(item).difference(X)
                    if Y:
                        X = frozenset(X)
                        XY = X | frozenset(Y)

                        # Checks if XY and X exists in the dictionary of frequent items
                        if XY in dic and X in dic:
                            confidence = dic[XY] / dic[X]

                            # Add {X} -> {Y} that have confidence above the min_confidence 
                            if confidence >= min_confidence:
                                rules.append((X, Y, confidence))
        return rules

    rules = association_rules(frequent_itemsets, min_confidence)

    # Stop timer
    end = time.time()

    # Write all itemsets and support to textfile, bruteforce will add all possible itemset combinations
    with open(f"{table}_bruteforce_unfilteredfreqlist_output.txt", "w") as f:
        f.write(f"min_support: {min_support}, min_confidence: {min_confidence}\n")
        
        for i in itemsets:
            item, supp = i
            f.write(f"{item}: {supp}\n")

    # Write all frequent itemsets and support to textfile
    with open(f"{table}_bruteforce_freqlist_output.txt", "w") as f:
        f.write(f"min_support: {min_support}, min_confidence: {min_confidence}\n")
        
        for i in frequent_itemsets:
            item, supp = i
            f.write(f"{item}: {supp}\n")

    # Write all association rules and confidence to textfile
    with open(f"{table}_bruteforce_association_output.txt", "w") as f:
        f.write(f"min_support: {min_support}, min_confidence: {min_confidence}\n")
        
        for i in rules:
            X, Y, conf = i
            f.write(f"{X} -> {Y} : {conf}\n")

    print("Brute Force")
    print("Runtime:", end - start, "seconds\n")
    return

try:
    brute_force(table)
except:
    print("There is an error, please try another combination of support, confidence, and df")

Brute Force
Runtime: 249.1199414730072 seconds



Apriori Algorithm Section

In [59]:
def apriori_algo(table):
    # Intialize db as a dataframe
    conn = sqlite3.connect(f'{table}.db')

    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)

    conn.close()

    print(df)

    df['Transactions'] = df['Transactions'].str.split(', ').apply(set)

    start = time.time()
    

    # Generate all k-itemsets using the combinations from [k-1]-itemsets starting from k=3
    def generate_new_combinations(prev_frequent_item, k):
        prev_frequent_item_list = list(prev_frequent_item)  # Convert set to list
        new_combinations = set()
        # Iterate over unique pairs of combinations
        for i, item_1 in enumerate(prev_frequent_item_list):
            for item_2 in prev_frequent_item_list[i + 1:]:
                # Prunes k-itemset that are not frequent in the previous level
                if all(x == y for x, y in zip(item_1[:-1], item_2[:-1])):
                    
                    # Create the k-itemset from the frequent 
                    union_set = tuple(sorted(set(item_1 + item_2)))
                    if len(union_set) == k:
                        new_combinations.add(union_set)
        return new_combinations

    # Generates all 2-itemsets from the frequent 1-itemset
    def generate_combinations(item, k):
        return set(combinations(item, k))

    # Calcualtes the support for a given itemset by dividing the frequency of the itemset in the transaction list by the total number of transactions
    def calculate_support(data, itemset):
        count = 0
        for row in data:
            if set(item for tpl in itemset for item in tpl).issubset(row):
                count += 1
        return count / df.shape[0]

    # Generate all frequent 1-itemset
    def first_pass(df, min_support):
        item = set()
        max_transaction_length = 1

        # Find all unique items in the transaction list
        for index, row in df.iterrows():
            for i in row['Transactions']:
                item.add(tuple([i]))
                max_transaction_length = max(len(row['Transactions']), max_transaction_length)

        frequent_itemsets = []
        remove = []

        # Checks if the unique items are frequent
        for i in item:
            support = calculate_support(df['Transactions'].values, tuple([i]))
            if support < min_support:
                remove.append(i)
            else:
                frequent_itemsets.append([[i], support])

        # Remove infrequent items 
        item.difference_update(remove)

        return item, frequent_itemsets, max_transaction_length

    # Generate all frequent k-itemset, where k >= 2
    def second_pass(df, item, frequent_itemsets, min_support, max_transaction_length):
        k = 2
        while True:
            # Generate k-itemsets
            if k == 2:
                all_combinations = generate_combinations(item, k)
            else:
                all_combinations = generate_new_combinations(next_combinations, k)
            next_combinations = set()
            # For every k-itemset, check if they are frequent
            for i in all_combinations:
                support = calculate_support(df['Transactions'].values, i)
                if support >= min_support:
                    frequent_itemsets.append([list(i), support])
                    # Adds only the frequent 
                    next_combinations.add(i)
            
            # If there are no more frequent itemsets, we know to stop looking at the next level
            if len(next_combinations) == 0:
                break
            
            k += 1

        return frequent_itemsets

    # Generate frequent 1-itemsets 
    item, frequent_itemsets, max_transaction_length = first_pass(df, min_support)

    # Generate frequent k-itemsets 
    frequent_itemsets = second_pass(df, item, frequent_itemsets, min_support, max_transaction_length)

    # Create all subset for an item in the frequent itemset
    # If item = A, B and item length is 2
    # The subsets would be A, B, AB
    def find_subset(item, item_length):
        subsets = []
        for i in range(1, item_length + 1):
            subsets.extend(combinations(item, i))
        return subsets

    # Generate all association rules from the frequent itemset
    def association_rules(frequent_itemsets, min_confidence):
        rules = list()
        dic = {}

        # Convert the frequent itemset into a dictionary
        for item in frequent_itemsets:
            key = frozenset(item[0])
            value = item[1]
            dic[key] = value

        # Generate all possible {X} -> {Y} and checks if {XY} exist to compute confidence
        for item, support in dic.items():
            item_length = len(item)
            if item_length > 1:
                subsets = find_subset(item, item_length)
                for X in subsets:
                    Y = set(item).difference(X)
                    if Y:
                        X = frozenset(X)
                        XY = X | frozenset(Y)

                        # Checks if XY and X exists in the dictionary of frequent items
                        if XY in dic and X in dic:
                            confidence = dic[XY] / dic[X]

                            # Add {X} -> {Y} that have confidence above the min_confidence 
                            if confidence >= min_confidence:
                                rules.append((X, Y, confidence))
        return rules

    # Generate all association rules above the minimum confidence
    rules = association_rules(frequent_itemsets, min_confidence)

    # Stop timer
    end = time.time()

    # Write all frequent itemsets and support to textfile
    with open(f"{table}_apriorialgo_freqlist_output.txt", "w") as f:
        f.write(f"min_support: {min_support}, min_confidence: {min_confidence}\n")
        
        for i in frequent_itemsets:
            item, supp = i
            f.write(f"{item}: {supp}\n")

    # Write all association rules and confidence to textfile
    with open(f"{table}_apriorialgo_association_output.txt", "w") as f:
        f.write(f"min_support: {min_support}, min_confidence: {min_confidence}\n")
        
        for i in rules:
            X, Y, conf = i
            f.write(f"{X} -> {Y} : {conf}\n")

    print("Apriori Algorithm")
    print("Runtime:", end - start, "seconds\n")
    return 

try:
    apriori_algo(table)
except:
    print("There is an error, please try another combination of support, confidence, and df")

    Transaction_ID                                       Transactions
0                1  Wonderful Pistachios No Shells, PASTA RONI Qua...
1                2                                Frito-Lay Party Mix
2                3  Quaker Instant Oatmeal Express Cups, Jack Link...
3                4  GoGo squeeZ Fruit on the Go, Nature Valley Cru...
4                5  Bumble Bee Chunk Light Tuna In Water, HORMEL C...
5                6  Mott Fruit Flavored Snacks, Nature Valley Crun...
6                7  Kelloggs Cold Breakfast Cereal, Chef Boyardee ...
7                8  TWIZZLERS Twists Strawberry Flavored Licorice ...
8                9  GoGo squeeZ Fruit on the Go, Maruchan Ramen Ch...
9               10  Kelloggs Cold Breakfast Cereal, TWIZZLERS Twis...
10              11  Quaker Instant Oatmeal, Quaker Instant Oatmeal...
11              12  SpaghettiOs Canned Pasta with Meatballs, Jack ...
12              13  Chef Boyardee Beef Ravioli, Premier Liquid Pro...
13              14  

Apriori Library Section

In [58]:
def apriori_library(table):
    
    # Intialize db as a dataframe
    conn = sqlite3.connect(f'{table}.db')

    df = pd.read_sql_query(f"SELECT * FROM {table}", conn)

    conn.close()

    # Start timer
    start = time.time()

    # One hot encode the unique items
    new_df = df['Transactions'].str.get_dummies(', ')

    # Generate frequent itemsets using mlxtend
    frequent_itemsets = apriori(new_df, min_support, use_colnames=True)

    # Generate association rules using mlxtend
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)

    # Stop timer
    end = time.time()

    # Write all frequent itemsets and support to textfile
    with open(f"{table}_apriorilib_freqlist_output.txt", "w") as f:
        f.write(f"min_support: {min_support}, min_confidence: {min_confidence}\n")

        for index, row in frequent_itemsets.iterrows():
            f.write(f"{row['itemsets']}: {row['support']}\n")

    # Write all association rules and confidence to textfile
    with open(f"{table}_apriorilib_association_output.txt", "w") as f:
        f.write(f"min_support: {min_support}, min_confidence: {min_confidence}\n")

        for index, rule in rules.iterrows():
            f.write(f"{rule['antecedents']} -> {rule['consequents']}: {rule['confidence']}\n")

    print("Apriori Algorithm from mlxtend")
    print("Runtime:", end - start, "seconds\n")
    return

try:
    apriori_library(table)
except:
    print("There is an error, please try another combination of support, confidence, and df")

Apriori Algorithm from mlxtend
Runtime: 0.010971546173095703 seconds



c:\Users\Christopher\AppData\Local\Programs\Python\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
